In [3]:
import sys

import numpy as np 
from argparse import ArgumentParser
from acas.acasxu_sim import AcasEnv, Airplane, DubinsAgent, LutAgent, ConstantAgent, RandomAgent, AidgeAgent, consolidate_agents, AcasRender
#from acas.acasxu_sim import run_single_sim

import matplotlib
print(matplotlib.rcsetup.interactive_bk)
%matplotlib tkagg 
#nbAgg #inline #notebook #widget
import matplotlib.pyplot as plt
plt.ion()
print(matplotlib.get_backend())


#x coordinate of each airplane
#y coordinate of each airplane
#heading angle (bearing) of each airplane
#velocity of each airplane
airplanes = [Airplane(0.0, 0.0, 0.0, 1080.0), Airplane(25000.0, 50000.0, -1., 1000.0)]

agents = [AidgeAgent(), ConstantAgent()]   # DubinsAgent(), LutAgent(), RandomAgent()
horizon = 120


total_time=horizon
airplanes = [Airplane(x=0.0, y=0.0, head=0.0, speed=1080.0), 
             Airplane(x=25000.0, y=50000.0, head=-np.pi/3, speed=1000.0)]
#agents=['dubins', 'dubins']
   
env  = AcasEnv(airplanes=airplanes, save_states=True)
   
#agents = consolidate_agents(agents)

obs = env.reset()
actions = [agent.reset(obs[i]) for i, agent in enumerate(agents)]

print("====================================================================")
print("RUNNING SIMULATION:")
print("--------------------------------------------------------------------")
print(f"rho (distance) = {env.rho[0]} ft")
print(f"theta (relative position angle) = {env.theta[0]} rad , = {env.theta[0]/np.pi} pi rad , = {np.degrees(env.theta[0])} deg")
print(f"psi (relative intrusion angle) = {env.psi[0]} rad , = {env.psi[0]/np.pi} pi rad , = {np.degrees(env.psi[0])} deg)")
print(f"v_own (own speed) = {env.airplanes[0].speed} ft/s")
print(f"v_int (intruder speed) = {env.airplanes[1].speed} ft/s.")
print(f"coords_own (x, y) = ({env.airplanes[0].x},{env.airplanes[0].y})")
print(f"heading_own (own absolute angle) = {env.airplanes[0].head} rad , = {env.airplanes[0].head/np.pi} pi rad , = {np.degrees(env.airplanes[0].head)} deg)")
print(f"coords_int (x, y) = ({env.airplanes[1].x},{env.airplanes[1].y})")
print(f"heading_int (intruder absolute angle) = {env.airplanes[1].head} rad , = {env.airplanes[1].head/np.pi} pi rad , = {np.degrees(env.airplanes[1].head)} deg)")
print("====================================================================")

for t in range(total_time):
   obs = env.step(actions=actions)
   actions = [m.react(obs[i]) for i, m in enumerate(agents)]

fig, ax = plt.subplots(figsize=(8,8))
renderer = AcasRender(env)
anim = renderer.plot(fig=fig, show=False)
plt.show()


ModuleNotFoundError: No module named 'acas.acasxu_sim'

In [2]:
import os

from scipy import ndimage
from scipy.linalg import expm

from matplotlib import patches, animation
from matplotlib.image import BboxImage
from matplotlib.transforms import Bbox, TransformedBbox
from matplotlib.collections import LineCollection
from matplotlib.path import Path
from matplotlib.lines import Line2D

from acas.lut_constants import get_act_idx, ACTION_NAMES, ACTION_CODES, ACT_IDX_COC


fig, ax = plt.subplots(figsize=(8,8))

ax.set_aspect('equal')
#ax.axis('equal')
        
ax.set_title("Simulation")
ax.set_xlabel('X Position (ft)')
ax.set_ylabel('Y Position (ft)')

palette = np.array(['gray', 'green', 'blue', 'magenta', 'red'])
custom_lines = [Line2D([0], [0], color=c, lw=1) for c in palette]
ax.legend(custom_lines, ACTION_NAMES, fontsize=12)#, loc='lower left')
    
num_airplanes = len(env.airplanes)
total_time = len(env.states_history)
H = np.array(env.states_history)
A = np.array(env.commands_history)
C = palette[A]

time_text = ax.text(0.02, 0.98, 'Time: 0 s', horizontalalignment='left', fontsize=12, verticalalignment='top', transform=ax.transAxes)
time_text.set_visible(True)

lines = []
lcs = []
figs = []

#airplane_img_filepath = os.path.dirname(os.path.realpath(__file__)) + '/img/airplane.png'
airplane_img_filepath = '../src/acas/img/airplane.png'
img = plt.imread(airplane_img_filepath)

#lc = LineCollection([], lw=2, animated=True, color='k', zorder=1)
#ax.add_collection(lc)

interval = 10

for i in range(num_airplanes):
    #initial state
    x, y, theta, speed = H[0,i]
    x_f, y_f, theta_f, speed_f = H[-1,i]
    #total displacement
    d = speed * total_time
    dx = d * np.cos(theta)
    dy = d * np.sin(theta)
    # draw linear initial trajectory
    ax.scatter([x, x_f], [y, y_f], zorder=3, alpha=0.3)
    ax.plot([x, x+dx], [y, y+dy], ls=':', lw=1, color='lightgray', zorder=2)
    l = ax.plot(x, y, ls='-', lw=2, zorder=1)[0]
    #l = Line2D(x, y, lc='c', lw=1, ls='-', zorder=1)
    lines.append(l)
    lc = LineCollection([], linestyle='solid', lw=2, animated=True, color='k', zorder=1)
    ax.add_collection(lc)
    lcs.append(lc)
    #planes
    size=1500 #5000
    box = Bbox.from_bounds(x - size/2, y - size/2, size, size)
    tbox = TransformedBbox(box, ax.transData)
    box_image = BboxImage(tbox, zorder=2)
    theta_deg = (theta - np.pi / 2) / np.pi * 180 # original image is facing up, not right
    img_rotated = ndimage.rotate(img, theta_deg, order=1)
    box_image.set_data(img_rotated)
    figs.append(box_image)
    ax.add_artist(box_image)
    
artists = figs + lines + lcs + [time_text]

def init():
    time_text.set_text('Time: 0 s')
    for i, lc in enumerate(lcs):
        lc.get_paths().clear()
    return artists

def animate(n):
    time_text.set_text(f'Time: {n} s')
    #for i, l in enumerate(lines):
    for i, lc in enumerate(lcs):
        #airplane image
        fig=figs[i]
        #trajectory using lines
        #l.set_xdata(H[:n+1,i,0])
        #l.set_ydata(H[:n+1,i,1])
        #trajectory using paths
        paths = lc.get_paths()
        paths.append(Path([H[n,i,:2], H[n+1,i,:2]]))
        lc.set_colors(C[:n+1,i])
        x = H[n+1,i,0]
        y = H[n+1,i,1]
        theta = H[n+1,i,2]
        #airplane figure
        theta_deg = (theta - np.pi / 2) / np.pi * 180 # original image is facing up, not right
        original_size = list(img.shape)
        img_rotated = ndimage.rotate(img, theta_deg, order=1)
        rotated_size = list(img_rotated.shape)
        ratios = [r / o for r, o in zip(rotated_size, original_size)]
        fig.set_data(img_rotated)
        size = 1500 #5000
        width = size * ratios[0]
        height = size * ratios[1]
        box = Bbox.from_bounds(x - width/2, y - height/2, width, height)
        tbox = TransformedBbox(box, ax.transData)
        fig.bbox = tbox
    return artists

anim = FuncAnimation(fig, animate, init_func=init, frames=total_time-1, interval=interval, blit=True, repeat=True, repeat_delay=100)

plt.show()


ModuleNotFoundError: No module named 'acas.lut_constants'